In [1]:
import numpy as np
from math import sqrt

In [2]:
data = np.loadtxt("data/spambase.data", dtype='f', delimiter=',')

print (data)

max_value_in_data = [ 0 ] * 58

for lin in data:
    for x in range(0, 58):
        max_value_in_data[x] = max( max_value_in_data[x], lin[x] )
        

[[0.000e+00 6.400e-01 6.400e-01 ... 6.100e+01 2.780e+02 1.000e+00]
 [2.100e-01 2.800e-01 5.000e-01 ... 1.010e+02 1.028e+03 1.000e+00]
 [6.000e-02 0.000e+00 7.100e-01 ... 4.850e+02 2.259e+03 1.000e+00]
 ...
 [3.000e-01 0.000e+00 3.000e-01 ... 6.000e+00 1.180e+02 0.000e+00]
 [9.600e-01 0.000e+00 0.000e+00 ... 5.000e+00 7.800e+01 0.000e+00]
 [0.000e+00 0.000e+00 6.500e-01 ... 5.000e+00 4.000e+01 0.000e+00]]


Isso mostra que teremos que tratar os dados antes de rodar __Online SVM via Online Gradient Descent__

Temos ao menos duas etapas:
1 - Transformar os valores de classe ( variável target ) de 0, 1 para -1 ,1
2 - Temos que fazer uma normalização dos parâmetros. Pois alguns dados são percentuais e estão no range [0, 100] e outros podem atingir valores maiores como por exemplo 15841. Não queremos que um atributo tenha uma importância maior que outro por ser representado em uma escala diferente. Vamos fazer uma normalização __hard__ que é mapear o maior valor de uma determinada feature para 1 e dividir os outros valores dessa feature específica pelo maior valor encontrado.

Se a normalização hard se mostrar ruim na prática, podemos pensar em outras normalizações. ( Soft usando mediana e desvio-padrão )..



In [3]:
# vamos alterar os valores da variável target de (0, 1) para (-1, 1)
for lin in data:
    if lin[57] == 0:
        lin[57] = -1
        
# Agora vamos fazer a normalizacao hard
normalized_data = data

for lin in data:
    for x in range(0, 57):
        max_value_in_data[x] = max( max_value_in_data[x], lin[x] )
        
for lin in normalized_data:
    for x in range(0, 57):
        lin[x] = (lin[x] / max_value_in_data[x] )
        

Breve descrição de Online SVM via Online Gradient Descent
* No instante t
    * Selecionamos um vetor pt
    * Recebemos uma nova instância do dataset (yt, zt), onde yt são os atributos e zt é a classe correspondente
    * Vamos tomar uma Hinge Loss definida como max(0, 1 - zt * <pt, yt> )
    * Utilizaremos o gradiente da função de perda do instante t para calcular o próximo p(t+1)
    
Vamos também utilizar sempre pt pertencente ao espaço euclideano de norma <= 1.

A princípio o p0 pode ser definido arbitrariamente, mas como vimos descrito em alguns lugares da literatura a inicialização com o vetor nulo vamos adotar essa estratégia.
   

In [4]:
# Aqui algumas funções auxiliares úteis
dimension = 57

def dot_product( a, b ):
    ans = 0
    for x in range(dimension):
        ans += ( a[x] * b[x] )
    return ans

def hinge_loss( p_t, z_t, y_t):
    return max(0, 1 - dot_product(p_t, y_t) * z_t )

def euclidean_norm( p_t ):
    ans = 0
    for coord in p_t:
        ans += (coord ** 2)
    return sqrt( ans )

def project_into_euclidean_norm_1( p_t ):
    norm = euclidean_norm( p_t )
    if norm <= 1:
        return p_t
    
    norm_p = p_t
    for coord in norm_p:
        coord = ( coord / norm )
    return norm_p

def get_gradient( y_t, z_t ):
    gradient = y_t
    for x in gradient:
        x *= ( - z_t )
    return gradient



In [5]:

# Vamos armazenar os pt obtidos ao longo do tempo, para poder analisar quão rápido o método alcança bons classificadores
# E no final vamos testar os valores obtidos como classificadores para o dataset completo e analisar a acurácia
all_p = []

current_p = [0] * 57

all_p.append( current_p )

# Temos algumas estratégias para definir o eta utilizado para atualizar p_t
# Vamos usar o eta proposto por Zinkevich que é da ordem de 1/sqrt(t)
# Que vai diminuindo conforme a quantidade de instancias processadas aumenta

# Se tivéssemos a segunda derivada da função de perda poderiamos usar 
# uma outra estratégia descoberta por Hazan em que eta_t = 1 / Ht, onde H é 
# um valor relacionado com a segunda derivada

# Essas estratégias para eta são definidas aqui (http://www.mit.edu/~rakhlin/papers/adaptive.pdf)


cumulative_loss = 0
# Agora vamos processar o dataset uma instancia de cada vez, num setup online

t = 1

for instance in normalized_data:
    instance_features = instance[0:57] # slices are semi-open intervals (:, [0, 57)
    target_class = instance[57]
    # vamos tomar a perda hinge relativo a current_p
    current_loss = hinge_loss( current_p, target_class, instance_features )
    cumulative_loss += current_loss
    
    if t % 100 == 0:
        print('perda na iteracao %d = %f' % (t, current_loss) )
    
    # Agora vamos usar o gradiente / subgradiente para fazer a atualizacao de p_(t + 1)
    # Se hinge_loss = 0, posso usar o subgradiente definido pela reta y = 0, e não mexer em p_t
    
    # O caso de fato interessante é quando a perda é positiva e queremos seguir no sentido contrário do subgradiente
    if current_loss != 0:
        gradient = get_gradient( instance_features, target_class )
        current_eta = 1 / sqrt(t)
        # p_t = p_(t - 1) - eta_t * ( gradiente da função de perda )
        for coord in range(57):
            current_p[coord] = (current_p[coord] - current_eta * gradient[coord])
        
        current_p = project_into_euclidean_norm_1( current_p )
            
        
    all_p.append( current_p ) # Adding current support vector to the list
    t += 1 # increment in one the number of processed instances


perda na iteracao 100 = 2.233652
perda na iteracao 200 = 1.971643
perda na iteracao 300 = 2.754252
perda na iteracao 400 = 2.506226
perda na iteracao 500 = 3.998994
perda na iteracao 600 = 3.742833
perda na iteracao 700 = 4.484138
perda na iteracao 800 = 4.962326
perda na iteracao 900 = 6.403390
perda na iteracao 1000 = 7.638121
perda na iteracao 1100 = 7.696410
perda na iteracao 1200 = 4.018514
perda na iteracao 1300 = 4.582764
perda na iteracao 1400 = 5.675093
perda na iteracao 1500 = 6.675964
perda na iteracao 1600 = 3.738295
perda na iteracao 1700 = 5.376902
perda na iteracao 1800 = 1.584158
perda na iteracao 1900 = 0.956205
perda na iteracao 2000 = 0.504747
perda na iteracao 2100 = 0.000000
perda na iteracao 2200 = 0.000000
perda na iteracao 2300 = 0.000000
perda na iteracao 2400 = 0.000000
perda na iteracao 2500 = 0.000000
perda na iteracao 2600 = 0.000000
perda na iteracao 2700 = 0.000000
perda na iteracao 2800 = 0.996728
perda na iteracao 2900 = 0.000000
perda na iteracao 3000 

Vamos agora __encapsular__ o que foi implementado acima em um método, para poder testar diversos parâmetros de __eta__. E também vamos fazer uma função que mede a acurácia de um determinado vetor __p__ para classificar os dados.



In [6]:
def online_svm(data, eta_fnc):
    current_p = [0] * 57
    all_p = [current_p]
    cumulative_loss = 0
    # Agora vamos processar o dataset uma instancia de cada vez, num setup online
    t = 1

    for instance in data:
        instance_features = instance[0:57] # slices are semi-open intervals (:, [0, 57)
        target_class = instance[57]
        # vamos tomar a perda hinge relativo a current_p
        current_loss = hinge_loss( current_p, target_class, instance_features )
        cumulative_loss += current_loss

        # Agora vamos usar o gradiente / subgradiente para fazer a atualizacao de p_(t + 1)
        # Se hinge_loss = 0, posso usar o subgradiente definido pela reta y = 0, e não mexer em p_t

        # O caso de fato interessante é quando a perda é positiva e queremos seguir no sentido contrário do subgradiente
        if current_loss != 0:
            gradient = get_gradient( instance_features, target_class )
            current_eta = eta_fnc( t ) # Eta calculado em funcao de t, de acordo com a funcao recebida como parametro
            
            # p_t = p_(t - 1) - eta_t * ( gradiente da função de perda )
            for coord in range(57):
                current_p[coord] = (current_p[coord] - current_eta * gradient[coord])

            current_p = project_into_euclidean_norm_1( current_p )


        all_p.append( current_p ) # Adding current support vector to the list
        t += 1 # increment in one the number of processed instances
    return (cumulative_loss, all_p)

Agora que encapsulamos o método em uma função que recebe como parâmetro uma função para o cálculo de eta, podemos começar a realizar alguns experimentos.

Antes disso, vou criar uma função que recebe um determinado vetor p, usa ele para __classificar__ todo o dataset e retorna a acurácia desse vetor.

In [17]:
def check_accuracy(data, classifier_vector):
    correct = 0
    total = 0
    for instance in data:
        instance_features = instance[0:57]
        target_class = instance[57]
        dot_prod = dot_product( instance_features, classifier_vector )
        predicted_class = -1
        if dot_prod >= 0:
            predicted_class = 1
        if target_class == predicted_class:
            correct += 1
        total += 1
    
    return ( correct / total )


def eta( t ):
    return 1 / sqrt(t)

def eta_constante( t ):
    return 0.5

results = online_svm( normalized_data, eta_constante )


melhor = 0
k = 0
for x in results[1]:
    if k % 100 == 0:
        #print(x)
        val = check_accuracy( normalized_data, x)
        if val > melhor:
            melhor = val
        print( val )
    k += 1
print(melhor)


0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
0.6059552271245382
